# Text Retrieval and Search Engines - Assignment 3
**Gal Noy** · 209346486

## 1. Setup & Dependencies

In [2]:
import pandas as pd
import json
import re
import string
from collections import Counter
from typing import List, Dict, Tuple, Optional, Literal
from tqdm.auto import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')

print("✓ Dependencies imported")

✓ Dependencies imported


### Install Required Packages

In [3]:
# !apt-get update
# !apt-get install -y openjdk-21-jdk
# !update-alternatives --install /usr/bin/java java /usr/lib/jvm/java-21-openjdk-amd64/bin/java 1
# !update-alternatives --install /usr/bin/javac javac /usr/lib/jvm/java-21-openjdk-amd64/bin/javac 1
# !update-alternatives --set java /usr/lib/jvm/java-21-openjdk-amd64/bin/java
# !update-alternatives --set javac /usr/lib/jvm/java-21-openjdk-amd64/bin/javac

In [4]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

!java -version

openjdk version "21.0.9" 2025-10-21
OpenJDK Runtime Environment (build 21.0.9+10-Ubuntu-122.04)
OpenJDK 64-Bit Server VM (build 21.0.9+10-Ubuntu-122.04, mixed mode, sharing)


In [5]:
# !pip install torch torchvision torchaudio
# !pip install pyserini==0.36.0
# !pip install accelerate
# !pip install transformers
# !pip install tqdm
# !pip install python-dotenv

### Hugging Face Authentication

In [6]:
from huggingface_hub import login
from dotenv import load_dotenv
import os

load_dotenv()

login(os.getenv('HUGGING_FACE_TOKEN'))
print("✓ Logged into Hugging Face")

✓ Logged into Hugging Face


## 2. Data Loading

In [7]:
df_train = pd.read_csv("./data/train.csv", converters={"answers": json.loads})
df_test = pd.read_csv("./data/test.csv")

print(f"Train set: {len(df_train)} questions")
print(f"Test set: {len(df_test)} questions")
print(f"\nSample question: {df_train.iloc[0]['question']}")
print(f"Sample answers: {df_train.iloc[0]['answers']}")

Train set: 3778 questions
Test set: 2032 questions

Sample question: what is the name of justin bieber brother?
Sample answers: ['Jazmyn Bieber', 'Jaxon Bieber']


## 3. Retrieval Functions

### Pyserini index

In [8]:
from pyserini.search import SimpleSearcher
from pyserini.index.lucene import IndexReader

print("Loading Pyserini index...")
searcher = SimpleSearcher.from_prebuilt_index('wikipedia-kilt-doc')
index_reader = IndexReader.from_prebuilt_index('wikipedia-kilt-doc')

print(f"✓ Index loaded: {index_reader.stats()['documents']} documents")

2025-12-20 00:41:47.481597844 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"


Loading Pyserini index...


Dec 20, 2025 12:41:48 AM org.apache.lucene.store.MemorySegmentIndexInputProvider <init>
INFO: Using MemorySegmentIndexInput with Java 21; to disable start with -Dorg.apache.lucene.store.MMapDirectory.enableMemorySegments=false


SimpleSearcher class has been deprecated, please use LuceneSearcher from pyserini.search.lucene instead
✓ Index loaded: 5903530 documents


### Retrieval manager

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from dataclasses import dataclass
from typing import List, Dict
from functools import lru_cache
import json

@lru_cache(maxsize=1000)
def get_doc_content(docid: str) -> str:
    """Return cached raw document text."""
    try:
        doc = searcher.doc(docid)
        return json.loads(doc.raw()).get("contents", "").replace("\n", " ")
    except Exception:
        return ""
    

@dataclass
class RetrievalManager:
    """
    Hybrid retrieval with:
      - BM25 document retrieval
      - QLD document retrieval
      - RRF on documents (recall stage)
      - Passage segmentation
      - Cross-encoder passage reranking (precision stage)
    """

    # Retrieval
    k_docs: int = 100
    k_passages: int = 7
    rrf_k: int = 60

    # BM25 / QLD
    mu: int = 1000
    k1: float = 0.9
    b: float = 0.4

    # Passage extraction
    window: int = 150
    overlap: int = 50
    min_passage_words: int = 30

    def __str__(self):
        return (
            f"RetrievalManager("
            f"k_docs={self.k_docs}, k_passages={self.k_passages}, "
            f"RRF_k={self.rrf_k}, "
            f"window={self.window}, overlap={self.overlap})"
        )

    def extract_passages(self, text: str) -> List[str]:
        """Split document text into overlapping word windows."""
        if not text:
            return []

        words = text.split()
        if len(words) < self.min_passage_words:
            return []

        step = max(1, self.window - self.overlap)
        passages = []

        for i in range(0, len(words), step):
            chunk = words[i:i + self.window]
            if len(chunk) < self.min_passage_words:
                break
            passages.append(" ".join(chunk))

        return passages

    def rerank_passages(self, query: str, passages: List[str]) -> List[str]:
        """
        Model-free embedding-based reranker using character n-gram TF-IDF
        and cosine similarity. Strongest distance-based method without
        pretrained models.
        """
        if not passages:
            return []

        vectorizer = TfidfVectorizer(
            analyzer="char",
            ngram_range=(3, 5),
            lowercase=True,
            min_df=1,
        )

        tfidf = vectorizer.fit_transform([query] + passages)
        query_vec = tfidf[0]
        passage_vecs = tfidf[1:]

        scores = cosine_similarity(query_vec, passage_vecs).flatten()

        ranked = sorted(
            zip(passages, scores),
            key=lambda x: x[1],
            reverse=True,
        )

        return [p for p, _ in ranked[:self.k_passages]]
    
    def retrieve_context(self, query: str) -> List[str]:
        """Return top answer-bearing passages."""

        # Lexical retrieval
        searcher.set_bm25(self.k1, self.b)
        bm25_docids = [h.docid for h in searcher.search(query, self.k_docs)]

        searcher.set_qld(self.mu)
        qld_docids = [h.docid for h in searcher.search(query, self.k_docs)]

        # RRF on documents
        doc_scores: Dict[str, float] = {}

        for rank, docid in enumerate(bm25_docids):
            doc_scores[docid] = doc_scores.get(docid, 0.0) + 1.0 / (self.rrf_k + rank + 1)

        for rank, docid in enumerate(qld_docids):
            doc_scores[docid] = doc_scores.get(docid, 0.0) + 1.0 / (self.rrf_k + rank + 1)

        ranked_docids = sorted(
            doc_scores,
            key=doc_scores.get,
            reverse=True,
        )

        # Passage extraction
        passages: List[str] = []

        for docid in ranked_docids:
            content = get_doc_content(docid)
            if not content:
                continue

            passages.extend(self.extract_passages(content))

            # implicit cap: enough passages for reranking
            if len(passages) >= self.k_docs * 5:
                break

        # Cross-encoder reranking
        return self.rerank_passages(query, passages)
    

query = "Who wrote Harry Potter?"

rm = RetrievalManager()
print(rm)

passages = rm.retrieve_context(query)
for i, p in enumerate(passages, 1):
    print(f"{i}. {p[:120]}...")


RetrievalManager(k_docs=100, k_passages=7, RRF_k=60, window=150, overlap=50)
1. also provided the voice of Ginny in the video game versions of "Harry Potter and the Order of the Phoenix"; "Harry Potte...
2. of initial plain Jane who keeps growing on you." Comingsoon.net wrote "that If someone told me eight years ago that Bonn...
3. on Terror. Section::::"Harry Potter" used by political activists. Section::::"Harry Potter" used by political activists....
4. studio album. He explained, "But maybe there's something like that [a "Deathly Hallows"-related full length] in the futu...
5. user GamerXC3. Since this secondary publishing in August 2012, the YouTube video has had more than 1.6 million views. Tw...
6. The Magical Worlds of Harry Potter The Magical Worlds of Harry Potter: A Treasury of Myths, Legends, and Fascinating Fac...
7. a pun on J. K. Rowling. ("K. B." is a Hungarian shorthand for "körülbelül", meaning "approximately", while Rotring is a ...


## 4. LLM Generation

### Load LLM model

In [10]:
import transformers
import torch
import logging

# Suppress transformers warnings
transformers.logging.set_verbosity_error()
logging.getLogger("transformers").setLevel(logging.ERROR)

print("Loading LLM model...")
model_id = "meta-llama/Llama-3.2-1B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.float16},
    device=0
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# Set pad_token for batch processing
pipeline.tokenizer.pad_token = pipeline.tokenizer.eos_token

print(f"✓ Model loaded on: {'GPU' if torch.cuda.is_available() else 'CPU'}")

Loading LLM model...
✓ Model loaded on: GPU


### Prompt manager

In [14]:
SYSTEM_PROMPT = (
    "You are a strict, grounded Question Answering system.\n"
    "You are given documents and a question.\n"
    "Answer ONLY using information that appears in the documents.\n"
    "Your answer must be ONLY the entity or value that answers the question.\n"
    "Do NOT return sentences, clauses, or descriptions.\n"
    "If the answer cannot be verified in the documents, return: unknown."
)


USER_PROMPT = (
    "Documents:\n"
    "{context}\n\n"
    "Task:\n"
    "- Answer the question using only the documents.\n"
    "- Return ONE single answer only (not a list or multiple items).\n"
    "- The answer must match the question type (person, place, date, number).\n"
    "- Return the shortest complete answer that answers the question.\n"
    "- The answer must appear verbatim or as a clear entity in the documents.\n"
    "- Do NOT return explanations, relations, or full sentences.\n"
    "- If you cannot verify the answer in the documents, output: unknown.\n\n"
    "Question: {question}\n"
    "Answer:"
)


@dataclass
class PromptManager:
    """Manages prompt generation and LLM answer generation."""
    system_prompt: str = SYSTEM_PROMPT
    user_prompt: str = USER_PROMPT
    temperature: float = 0.0
    top_p: float = 1.0
    max_new_tokens: int = 64
    do_sample: bool = False
    
    def __str__(self):
        return f"temp={self.temperature}, top_p={self.top_p}, max_tokens={self.max_new_tokens}"

    @staticmethod
    def clean_answer(answer: str) -> str:
        """Clean and standardize the generated answer."""
        answer = re.sub(r'^(Answer|The answer is|Based on the .*?,):?\s*', '', answer, flags=re.I)
        answer = answer.rstrip('.')
        if any(phrase in answer.lower() for phrase in ["dont know", "don't know", "do not know", "unknown"]):
            return "unknown"
        return answer.strip()

    def create_messages(self, question: str, contexts: List[str]) -> List[Dict]:
        """Create messages for the LLM based on the question and contexts."""
        if not contexts:
            context_str = "No relevant documents found."
        else:
            context_str = '\n\n'.join([f"Document {i+1}: {ctx}" for i, ctx in enumerate(contexts)])
        
        return [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": self.user_prompt.format(context=context_str, question=question)}
        ]

    def generate_answer(self, question: str, contexts: List[str]) -> str:
        """Generate an answer using the LLM based on the question and contexts."""
        messages = self.create_messages(question, contexts)
        
        outputs = pipeline(
            messages,
            max_new_tokens=self.max_new_tokens,
            eos_token_id=terminators,
            do_sample=self.do_sample,
            temperature=self.temperature,
            top_p=self.top_p
        )
        
        answer = outputs[0]["generated_text"][-1].get('content', '')
        return self.clean_answer(answer)


test_prompt_manager = PromptManager()
print(f"Testing: {test_prompt_manager}")
test_answer = test_prompt_manager.generate_answer(query, passages)
print(f"✓ Generated answer: '{test_answer}'")

Testing: temp=0.0, top_p=1.0, max_tokens=64
✓ Generated answer: 'J. K. Rowling'


## 5. Evaluation Metrics

In [15]:
def normalize_answer(s: str) -> str:
    """Normalize answer for comparison"""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    
    def white_space_fix(text):
        return ' '.join(text.split())
    
    def remove_punc(text):
        return ''.join(ch for ch in text if ch not in set(string.punctuation))
    
    def lower(text):
        return text.lower()
    
    return white_space_fix(remove_articles(remove_punc(lower(s))))


def compute_token_metrics(prediction: str, ground_truth: str) -> Tuple[float, float, float]:
    """
    Compute precision, recall, and F1 score for token-level comparison.
    Returns: (precision, recall, f1)
    """
    pred_tokens = normalize_answer(prediction).split()
    gt_tokens = normalize_answer(ground_truth).split()
    
    # Handle empty cases
    if len(pred_tokens) == 0 or len(gt_tokens) == 0:
        match = int(pred_tokens == gt_tokens)
        return match, match, match
    
    # Compute overlap
    common = Counter(pred_tokens) & Counter(gt_tokens)
    num_same = sum(common.values())
    
    if num_same == 0:
        return 0.0, 0.0, 0.0
    
    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    
    return precision, recall, f1


def evaluate_predictions(df_gold: pd.DataFrame, predictions: Dict[int, str]) -> Dict:
    """Evaluate predictions against ground truth."""
    f1_scores = []
    precision_scores = []
    recall_scores = []
    exact_matches = []
    
    for _, row in df_gold.iterrows():
        qid = row['id']
        
        # Handle missing predictions
        if qid not in predictions:
            f1_scores.append(0.0)
            precision_scores.append(0.0)
            recall_scores.append(0.0)
            exact_matches.append(0)
            continue
        
        prediction = predictions[qid]
        ground_truths = row['answers']
        
        # Normalize once
        norm_prediction = normalize_answer(prediction)
        
        # Find best match across all ground truths
        best_f1 = 0.0
        best_precision = 0.0
        best_recall = 0.0
        is_exact = 0
        
        for gt in ground_truths:
            norm_gt = normalize_answer(gt)
            
            # Compute metrics
            prec, rec, f1 = compute_token_metrics(prediction, gt)
            
            # Track best scores
            if f1 > best_f1:
                best_f1 = f1
                best_precision = prec
                best_recall = rec
            
            # Check exact match
            if norm_prediction == norm_gt:
                is_exact = 1
        
        f1_scores.append(best_f1)
        precision_scores.append(best_precision)
        recall_scores.append(best_recall)
        exact_matches.append(is_exact)
    
    return {
        'f1': 100.0 * sum(f1_scores) / len(f1_scores) if f1_scores else 0.0,
        'precision': 100.0 * sum(precision_scores) / len(precision_scores) if precision_scores else 0.0,
        'recall': 100.0 * sum(recall_scores) / len(recall_scores) if recall_scores else 0.0,
        'exact_match': 100.0 * sum(exact_matches) / len(exact_matches) if exact_matches else 0.0,
        'f1_scores': f1_scores,
        'precision_scores': precision_scores,
        'recall_scores': recall_scores,
        'exact_matches': exact_matches
    }


# Test evaluation
test_predictions = {1: "J.K. Rowling", 2: "Paris", 3: "Shakespeare"}
test_gold = pd.DataFrame({
    'id': [1, 2, 3],
    'answers': [["J.K. Rowling", "Rowling"], ["Earth"], ["William Shakespeare", "Shakespeare"]]
})

test_metrics = evaluate_predictions(test_gold, test_predictions)
print(f"✓ Evaluation test: F1={test_metrics['f1']:.2f}, P={test_metrics['precision']:.2f}, R={test_metrics['recall']:.2f}, EM={test_metrics['exact_match']:.2f}")

✓ Evaluation test: F1=66.67, P=66.67, R=66.67, EM=66.67


## 6. Experiment Framework

In [16]:
def run_experiment(
    name: str,
    df_data: pd.DataFrame,
    retrieval_manager: RetrievalManager,
    prompt_manager: PromptManager,
    max_questions: Optional[int] = None,
    verbose: bool = True
) -> Dict:
    """
    Run a full experiment: retrieval + prompting + evaluation.
    Args:
        name: Name of the experiment.
        df_data: DataFrame with questions and answers.
        retrieval_manager: RetrievalManager instance.
        prompt_manager: PromptManager instance.
        max_questions: Optional limit on number of questions to process.
        verbose: Whether to print progress and results.
    """
    if max_questions:
        df_data = df_data.head(max_questions)

    predictions = {}

    iterator = tqdm(df_data.iterrows(), total=len(df_data), desc=name) if verbose else df_data.iterrows()

    for _, row in iterator:
        question = row['question']
        qid = row['id']

        contexts = retrieval_manager.retrieve_context(question)
        answer = prompt_manager.generate_answer(question, contexts)

        predictions[qid] = answer

    metrics = evaluate_predictions(df_data, predictions)

    result = {
        'name': name,
        'retrieval': retrieval_manager,
        'prompt': prompt_manager,
        'f1_score': metrics['f1'],
        'precision': metrics['precision'],
        'recall': metrics['recall'],
        'exact_match': metrics['exact_match'],
        'num_questions': len(df_data),
        'predictions': predictions,
        'f1_scores': metrics['f1_scores'],
        'precision_scores': metrics['precision_scores'],
        'recall_scores': metrics['recall_scores'],
        'exact_matches': metrics['exact_matches']
    }

    if verbose:
        print(f"\n{name}")
        print(f"   Retrieval: {retrieval_manager}")
        print(f"   Prompt: {prompt_manager}")
        print(
            f"   F1={metrics['f1']:.2f} | "
            f"P={metrics['precision']:.2f} | "
            f"R={metrics['recall']:.2f} | "
            f"EM={metrics['exact_match']:.2f}"
        )
        print(f"   Questions: {len(df_data)}\n")

    return result


test_retrieval = RetrievalManager()
test_prompt = PromptManager()
print(f"Testing experiment with:")
print(f"  Retrieval: {test_retrieval}")
print(f"  Prompt: {test_prompt}")

test_exp = run_experiment(
    "Quick Test",
    df_train.head(25),
    test_retrieval,
    test_prompt,
    verbose=True
)

print(f"✓ Experiment framework ready")

Testing experiment with:
  Retrieval: RetrievalManager(k_docs=100, k_passages=7, RRF_k=60, window=150, overlap=50)
  Prompt: temp=0.0, top_p=1.0, max_tokens=64


Quick Test: 100%|██████████| 25/25 [00:30<00:00,  1.22s/it]


Quick Test
   Retrieval: RetrievalManager(k_docs=100, k_passages=7, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=27.98 | P=31.78 | R=30.73 | EM=8.00
   Questions: 25

✓ Experiment framework ready


## 7. Experiments

### Experiments global config

In [17]:
EXPERIMENT_SEED = 42
EXPERIMENTS_NUM_QUESTIONS = 100
DEFAULT_PROMPT_MANAGER = PromptManager()

### Experiments utils

In [18]:
def get_experiment_log_path(num_questions: int) -> str:
    """Get path for experiment log CSV based on number of questions."""
    return f"./results/grid_search_results_q{num_questions}.csv"


def generate_config_key(
    retrieval_mgr: RetrievalManager,
    prompt_mgr: PromptManager,
) -> str:
    """Generate unique config key for RRF-based retrieval."""
    return (
        f"RRF_k{retrieval_mgr.rrf_k}_"
        f"mu{retrieval_mgr.mu}_"
        f"k1{retrieval_mgr.k1}_b{retrieval_mgr.b}_"
        f"kdocs{retrieval_mgr.k_docs}_"
        f"kpass{retrieval_mgr.k_passages}_"
        f"win{retrieval_mgr.window}_ovl{retrieval_mgr.overlap}"
    )
    
    
def build_retrieval_manager(base: dict, override: dict) -> RetrievalManager:
    """Build RetrievalManager safely."""
    return RetrievalManager(**{**base, **override})


def save_results_to_csv(result: dict, key: str, path: str):
    """Save experiment results to a CSV file."""
    os.makedirs(os.path.dirname(path), exist_ok=True)

    row = {
        "config_key": key,
        "f1": result["f1_score"],
        "precision": result["precision"],
        "recall": result["recall"],
        "exact_match": result["exact_match"],
        "num_questions": result["num_questions"],
    }

    df = pd.DataFrame([row])
    if not os.path.exists(path):
        df.to_csv(path, index=False)
    else:
        df.to_csv(path, mode="a", header=False, index=False)


def load_completed_configs(path: str) -> set[str]:
    """Load set of completed experiment config keys from CSV."""
    if not os.path.exists(path):
        return set()
    return set(pd.read_csv(path)["config_key"])


### Print utils

In [19]:
def print_grid_results_table(grid: list[dict], *, num_questions: int):
    """Print results table for a given experiment grid."""
    pd.set_option("display.max_colwidth", None)

    path = get_experiment_log_path(num_questions)
    if not os.path.exists(path):
        print("No results file found.")
        return

    df = pd.read_csv(path)

    keys = [
        generate_config_key(g["retrieval_mgr"], g["prompt_mgr"])
        for g in grid
    ]

    grid_df = (
        df[df["config_key"].isin(keys)]
        .sort_values("f1", ascending=False)
        .reset_index(drop=True)
    )

    if grid_df.empty:
        print("No completed configs found for this grid.")
        return

    display_cols = [
        "config_key",
        "f1",
        "precision",
        "recall",
        "exact_match",
    ]

    print("\nGrid results (sorted by F1):")
    display(grid_df[display_cols])


### Best-config selector

In [20]:
def select_top_k_configs(
    retrieval_managers: list[RetrievalManager],
    prompt_managers: list[PromptManager],
    *,
    num_questions: int,
    top_k: int = 5,
):
    """Select top-k configurations from experiment results."""
    path = get_experiment_log_path(num_questions)
    df = pd.read_csv(path)

    scored_entries = []

    for r_mgr, p_mgr in zip(retrieval_managers, prompt_managers):
        key = generate_config_key(r_mgr, p_mgr)
        row = df[df["config_key"] == key]
        if row.empty:
            continue

        scored_entries.append({
            "retrieval_mgr": r_mgr,
            "prompt_mgr": p_mgr,
            "f1": float(row.iloc[0]["f1"]),
            "config_key": key,
        })

    scored_entries.sort(
        key=lambda x: (x["f1"], x["config_key"]),
        reverse=True,
    )

    return scored_entries[:top_k]


### Phase runner

In [21]:
def run_phase(
    *,
    phase_name: str,
    grid: list[dict],
    df_train: pd.DataFrame,
    num_questions: int,
    top_k: int | None = None,
):
    """
    Run a phase of experiments over a grid of configurations.
    Args:
        phase_name: Name of the experiment phase.
        grid: List of configurations (dicts with 'retrieval_mgr' and 'prompt_mgr').
        df_train: DataFrame with training questions and answers.
        num_questions: Number of questions to use per configuration.
        top_k: If specified, return only the top-k configurations after running.
    """
    print("\n" + "=" * 80)
    print(phase_name)
    print(f"Questions per config: {num_questions}")
    print("=" * 80)

    path = get_experiment_log_path(num_questions)

    validation_data = df_train.sample(
        n=num_questions,
        random_state=EXPERIMENT_SEED,
    ).reset_index(drop=True)

    completed = load_completed_configs(path)

    pending = [
        g for g in grid
        if generate_config_key(g["retrieval_mgr"], g["prompt_mgr"]) not in completed
    ]

    print(f"Total configs: {len(grid)}")
    print(f"Completed: {len(grid) - len(pending)}")
    print(f"Pending: {len(pending)}")
    print("-" * 80)

    for i, entry in enumerate(pending, start=1):
        r_mgr = entry["retrieval_mgr"]
        p_mgr = entry["prompt_mgr"]
        key = generate_config_key(r_mgr, p_mgr)

        print(f"[{i}/{len(pending)}] Running: {key}")

        result = run_experiment(
            name=key,
            df_data=validation_data,
            retrieval_manager=r_mgr,
            prompt_manager=p_mgr,
            verbose=True,
        )

        save_results_to_csv(result, key, path)
        print(f"✓ F1={result['f1_score']:.4f}")

    print_grid_results_table(grid, num_questions=num_questions)

    if top_k is None:
        return grid

    return select_top_k_configs(
        [g["retrieval_mgr"] for g in grid],
        [g["prompt_mgr"] for g in grid],
        num_questions=num_questions,
        top_k=top_k,
    )


### Phase 1 - Capacity (k_docs / k_passages)

In [22]:
PHASE_1_GRID = []

BASE_RETRIEVAL_PARAMS = {
    "window": 150,
    "overlap": 50,
    "mu": 1000,
    "k1": 0.9,
    "b": 0.4,
}

K_DOCS = [10, 50, 100, 250, 500]
K_PASSAGES = [5, 7, 10]

CAPACITY_PAIRS = [
    (k_docs, k_passages)
    for k_docs in K_DOCS
    for k_passages in K_PASSAGES
]

for k_docs, k_passages in CAPACITY_PAIRS:
    PHASE_1_GRID.append({
        "retrieval_mgr": RetrievalManager(
            k_docs=k_docs,
            k_passages=k_passages,
            **BASE_RETRIEVAL_PARAMS,
        ),
        "prompt_mgr": DEFAULT_PROMPT_MANAGER,
    })

PHASE_1_TOP_CONFIGS = run_phase(
    phase_name="PHASE 1 — Retrieval Capacity",
    grid=PHASE_1_GRID,
    df_train=df_train,
    num_questions=EXPERIMENTS_NUM_QUESTIONS,
    top_k=3,
)



PHASE 1 — Retrieval Capacity
Questions per config: 100
Total configs: 15
Completed: 0
Pending: 15
--------------------------------------------------------------------------------
[1/15] Running: RRF_k60_mu1000_k10.9_b0.4_kdocs10_kpass5_win150_ovl50


RRF_k60_mu1000_k10.9_b0.4_kdocs10_kpass5_win150_ovl50: 100%|██████████| 100/100 [00:45<00:00,  2.19it/s]



RRF_k60_mu1000_k10.9_b0.4_kdocs10_kpass5_win150_ovl50
   Retrieval: RetrievalManager(k_docs=10, k_passages=5, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=18.59 | P=20.26 | R=19.52 | EM=8.00
   Questions: 100

✓ F1=18.5881
[2/15] Running: RRF_k60_mu1000_k10.9_b0.4_kdocs10_kpass7_win150_ovl50


RRF_k60_mu1000_k10.9_b0.4_kdocs10_kpass7_win150_ovl50: 100%|██████████| 100/100 [00:50<00:00,  1.97it/s]



RRF_k60_mu1000_k10.9_b0.4_kdocs10_kpass7_win150_ovl50
   Retrieval: RetrievalManager(k_docs=10, k_passages=7, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=18.84 | P=20.29 | R=20.02 | EM=8.00
   Questions: 100

✓ F1=18.8391
[3/15] Running: RRF_k60_mu1000_k10.9_b0.4_kdocs10_kpass10_win150_ovl50


...

RRF_k60_mu1000_k10.9_b0.4_kdocs500_kpass7_win150_ovl50: 100%|██████████| 100/100 [07:12<00:00,  4.32s/it]



RRF_k60_mu1000_k10.9_b0.4_kdocs500_kpass7_win150_ovl50
   Retrieval: RetrievalManager(k_docs=500, k_passages=7, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=20.62 | P=24.05 | R=20.27 | EM=10.00
   Questions: 100

✓ F1=20.6159
[15/15] Running: RRF_k60_mu1000_k10.9_b0.4_kdocs500_kpass10_win150_ovl50


RRF_k60_mu1000_k10.9_b0.4_kdocs500_kpass10_win150_ovl50: 100%|██████████| 100/100 [07:26<00:00,  4.46s/it]


RRF_k60_mu1000_k10.9_b0.4_kdocs500_kpass10_win150_ovl50
   Retrieval: RetrievalManager(k_docs=500, k_passages=10, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=22.02 | P=25.00 | R=21.68 | EM=11.00
   Questions: 100

✓ F1=22.0242

Grid results (sorted by F1):


,config_key,f1,precision,recall,exact_match
0,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50,30.853361,36.066667,29.433333,20.0
1,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass5_win150_ovl50,28.135709,30.327778,29.850000,17.0
2,RRF_k60_mu1000_k10.9_b0.4_kdocs50_kpass10_win150_ovl50,25.651587,29.633333,25.100000,14.0
3,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass7_win150_ovl50,25.102381,28.583333,24.766667,15.0
4,RRF_k60_mu1000_k10.9_b0.4_kdocs250_kpass10_win150_ovl50,24.581746,27.038095,25.159524,15.0
5,RRF_k60_mu1000_k10.9_b0.4_kdocs50_kpass7_win150_ovl50,24.383664,28.455556,24.516667,13.0
6,RRF_k60_mu1000_k10.9_b0.4_kdocs250_kpass5_win150_ovl50,22.895238,25.045238,24.766667,11.0
7,RRF_k60_mu1000_k10.9_b0.4_kdocs500_kpass10_win150_ovl50,22.024206,24.995238,21.683333,11.0
8,RRF_k60_mu1000_k10.9_b0.4_kdocs50_kpass5_win150_ovl50,22.021429,25.066667,22.350000,11.0
9,RRF_k60_mu1000_k10.9_b0.4_kdocs250_kpass7_win150_ovl50,21.534127,23.616667,22.242857,11.0


### Phase 2 - Passage segmentation (window / overlap)

In [23]:
PHASE_2_GRID = []

WINDOW_OVERLAP_PAIRS = [
    (100, 30),
    (150, 50),
    (200, 50),
    (250, 60),
]

for entry in PHASE_1_TOP_CONFIGS:
    base = entry["retrieval_mgr"]
    for w, o in WINDOW_OVERLAP_PAIRS:
        PHASE_2_GRID.append({
            "retrieval_mgr": RetrievalManager(
                k_docs=base.k_docs,
                k_passages=base.k_passages,
                window=w,
                overlap=o,
                mu=base.mu,
                k1=base.k1,
                b=base.b,
            ),
            "prompt_mgr": DEFAULT_PROMPT_MANAGER,
        })

PHASE_2_TOP_CONFIGS = run_phase(
    phase_name="PHASE 2 — Passage Segmentation",
    grid=PHASE_2_GRID,
    df_train=df_train,
    num_questions=EXPERIMENTS_NUM_QUESTIONS,
    top_k=3,
)



PHASE 2 — Passage Segmentation
Questions per config: 100
Total configs: 12
Completed: 3
Pending: 9
--------------------------------------------------------------------------------
[1/9] Running: RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win100_ovl30


RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win100_ovl30: 100%|██████████| 100/100 [01:34<00:00,  1.06it/s]



RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win100_ovl30
   Retrieval: RetrievalManager(k_docs=100, k_passages=10, RRF_k=60, window=100, overlap=30)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=27.09 | P=30.13 | R=27.38 | EM=16.00
   Questions: 100

✓ F1=27.0913
[2/9] Running: RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win200_ovl50


RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win200_ovl50: 100%|██████████| 100/100 [02:46<00:00,  1.66s/it]



RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win200_ovl50
   Retrieval: RetrievalManager(k_docs=100, k_passages=10, RRF_k=60, window=200, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=25.04 | P=28.27 | R=23.93 | EM=17.00
   Questions: 100

✓ F1=25.0357
[3/9] Running: RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60


...

RRF_k60_mu1000_k10.9_b0.4_kdocs50_kpass10_win200_ovl50: 100%|██████████| 100/100 [01:55<00:00,  1.15s/it]



RRF_k60_mu1000_k10.9_b0.4_kdocs50_kpass10_win200_ovl50
   Retrieval: RetrievalManager(k_docs=50, k_passages=10, RRF_k=60, window=200, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=26.53 | P=30.55 | R=25.93 | EM=15.00
   Questions: 100

✓ F1=26.5294
[9/9] Running: RRF_k60_mu1000_k10.9_b0.4_kdocs50_kpass10_win250_ovl60


RRF_k60_mu1000_k10.9_b0.4_kdocs50_kpass10_win250_ovl60: 100%|██████████| 100/100 [02:34<00:00,  1.54s/it]


RRF_k60_mu1000_k10.9_b0.4_kdocs50_kpass10_win250_ovl60
   Retrieval: RetrievalManager(k_docs=50, k_passages=10, RRF_k=60, window=250, overlap=60)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=27.25 | P=31.58 | R=26.85 | EM=16.00
   Questions: 100

✓ F1=27.2468

Grid results (sorted by F1):


,config_key,f1,precision,recall,exact_match
0,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50,30.853361,36.066667,29.433333,20.0
1,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60,29.379004,32.895238,30.100000,17.0
2,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass5_win150_ovl50,28.135709,30.327778,29.850000,17.0
3,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass5_win200_ovl50,27.338889,29.783333,27.326190,18.0
4,RRF_k60_mu1000_k10.9_b0.4_kdocs50_kpass10_win250_ovl60,27.246825,31.578571,26.850000,16.0
5,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win100_ovl30,27.091270,30.133333,27.383333,16.0
6,RRF_k60_mu1000_k10.9_b0.4_kdocs50_kpass10_win200_ovl50,26.529365,30.550000,25.933333,15.0
7,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass5_win250_ovl60,26.246429,29.116667,26.600000,16.0
8,RRF_k60_mu1000_k10.9_b0.4_kdocs50_kpass10_win150_ovl50,25.651587,29.633333,25.100000,14.0
9,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win200_ovl50,25.035714,28.266667,23.933333,17.0


### Phase 3 - BM25 / QLD parameters

In [24]:
PHASE_3_GRID = []

BM25_PARAMS = [
    {"k1": 0.6, "b": 0.3},
    {"k1": 0.9, "b": 0.4},
    {"k1": 1.2, "b": 0.6},
]

QLD_PARAMS = [
    {"mu": 1000},
    {"mu": 2000},
]

for entry in PHASE_2_TOP_CONFIGS:
    base = entry["retrieval_mgr"]
    for bm25 in BM25_PARAMS:
        for qld in QLD_PARAMS:
            PHASE_3_GRID.append({
                "retrieval_mgr": RetrievalManager(
                    k_docs=base.k_docs,
                    k_passages=base.k_passages,
                    window=base.window,
                    overlap=base.overlap,
                    k1=bm25["k1"],
                    b=bm25["b"],
                    mu=qld["mu"],
                ),
                "prompt_mgr": DEFAULT_PROMPT_MANAGER,
            })

PHASE_3_TOP_CONFIGS = run_phase(
    phase_name="PHASE 3 — Lexical Hyperparameters",
    grid=PHASE_3_GRID,
    df_train=df_train,
    num_questions=EXPERIMENTS_NUM_QUESTIONS,
    top_k=None,
)



PHASE 3 — Lexical Hyperparameters
Questions per config: 100
Total configs: 18
Completed: 3
Pending: 15
--------------------------------------------------------------------------------
[1/15] Running: RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50


RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50: 100%|██████████| 100/100 [02:10<00:00,  1.31s/it]



RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50
   Retrieval: RetrievalManager(k_docs=100, k_passages=10, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=28.06 | P=31.66 | R=28.02 | EM=16.00
   Questions: 100

✓ F1=28.0551
[2/15] Running: RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50


RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50: 100%|██████████| 100/100 [02:11<00:00,  1.31s/it]



RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50
   Retrieval: RetrievalManager(k_docs=100, k_passages=10, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=28.07 | P=31.90 | R=27.27 | EM=16.00
   Questions: 100

✓ F1=28.0745
[3/15] Running: RRF_k60_mu2000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50


...

RRF_k60_mu1000_k11.2_b0.6_kdocs100_kpass5_win150_ovl50: 100%|██████████| 100/100 [01:52<00:00,  1.13s/it]



RRF_k60_mu1000_k11.2_b0.6_kdocs100_kpass5_win150_ovl50
   Retrieval: RetrievalManager(k_docs=100, k_passages=5, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=21.69 | P=23.05 | R=23.93 | EM=13.00
   Questions: 100

✓ F1=21.6872
[15/15] Running: RRF_k60_mu2000_k11.2_b0.6_kdocs100_kpass5_win150_ovl50


RRF_k60_mu2000_k11.2_b0.6_kdocs100_kpass5_win150_ovl50: 100%|██████████| 100/100 [01:54<00:00,  1.14s/it]


RRF_k60_mu2000_k11.2_b0.6_kdocs100_kpass5_win150_ovl50
   Retrieval: RetrievalManager(k_docs=100, k_passages=5, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=22.78 | P=24.20 | R=25.18 | EM=13.00
   Questions: 100

✓ F1=22.7753

Grid results (sorted by F1):


,config_key,f1,precision,recall,exact_match
0,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50,30.853361,36.066667,29.433333,20.0
1,RRF_k60_mu2000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50,29.424790,34.333333,28.766667,17.0
2,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60,29.379004,32.895238,30.100000,17.0
3,RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win250_ovl60,28.220028,32.695238,27.933333,17.0
4,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass5_win150_ovl50,28.135709,30.327778,29.850000,17.0
5,RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50,28.074521,31.903704,27.266667,16.0
6,RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50,28.055077,31.658466,28.016667,16.0
7,RRF_k60_mu2000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60,27.704004,31.866667,28.266667,17.0
8,RRF_k60_mu2000_k10.9_b0.4_kdocs100_kpass5_win150_ovl50,27.622613,30.994444,29.100000,16.0
9,RRF_k60_mu2000_k11.2_b0.6_kdocs100_kpass10_win150_ovl50,26.952568,30.700000,27.016667,16.0


### Phase 4 - Best configs comparison over more questions

In [25]:
PHASE_4_F1_RATIO = 0.90          # promote configs within 90% of best
PHASE_4_MAX = 10                 # safety cap
PHASE_4_NUM_QUESTIONS = 300

experiments_path = get_experiment_log_path(
    num_questions=EXPERIMENTS_NUM_QUESTIONS
)
experiments_df = pd.read_csv(experiments_path)

assert not experiments_df.empty, "No results found for experiments"

# Sort globally by F1
experiments_df = experiments_df.sort_values(
    "f1", ascending=False
).reset_index(drop=True)

best_f1 = experiments_df.iloc[0]["f1"]
threshold_f1 = best_f1 * PHASE_4_F1_RATIO

print(f"Best F1 ({EXPERIMENTS_NUM_QUESTIONS}q): {best_f1:.4f}")
print(f"Promotion threshold: {threshold_f1:.4f} "
      f"({PHASE_4_F1_RATIO:.0%} of best)")

# Promote configs close by ratio
promoted_df = (
    experiments_df[experiments_df["f1"] >= threshold_f1]
    .head(PHASE_4_MAX)
    .reset_index(drop=True)
)

print(f"\nPromoted configs: {len(promoted_df)}")
for i, row in promoted_df.iterrows():
    print(
        f"{i+1}. {row['config_key']} | "
        f"F1={row['f1']:.4f}"
    )


Best F1 (100q): 30.8534
Promotion threshold: 27.7680 (90% of best)

Promoted configs: 7
1. RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50 | F1=30.8534
2. RRF_k60_mu2000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50 | F1=29.4248
3. RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60 | F1=29.3790
4. RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win250_ovl60 | F1=28.2200
5. RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass5_win150_ovl50 | F1=28.1357
6. RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50 | F1=28.0745
7. RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50 | F1=28.0551


In [26]:
# Collect all unique configs by config_key
ALL_CONFIGS = {}

for entry in PHASE_1_GRID + PHASE_2_GRID + PHASE_3_GRID:
    key = generate_config_key(
        entry["retrieval_mgr"],
        entry["prompt_mgr"],
    )
    ALL_CONFIGS[key] = entry   # dedupe by key

# Build Phase 4 grid directly from promoted config_keys
PHASE_4_GRID = [
    {
        "retrieval_mgr": ALL_CONFIGS[key]["retrieval_mgr"],
        "prompt_mgr": DEFAULT_PROMPT_MANAGER,
    }
    for key in promoted_df["config_key"].values
    if key in ALL_CONFIGS
]

print(f"\n✓ Phase 4 grid size: {len(PHASE_4_GRID)}")

PHASE_4_RESULTS = run_phase(
    phase_name="PHASE 4 — Stabilized Comparison",
    grid=PHASE_4_GRID,
    df_train=df_train,
    num_questions=PHASE_4_NUM_QUESTIONS,
    top_k=None,
)



✓ Phase 4 grid size: 7

PHASE 4 — Stabilized Comparison
Questions per config: 300
Total configs: 7
Completed: 0
Pending: 7
--------------------------------------------------------------------------------
[1/7] Running: RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50


RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50: 100%|██████████| 300/300 [06:37<00:00,  1.33s/it]



RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50
   Retrieval: RetrievalManager(k_docs=100, k_passages=10, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=23.57 | P=27.14 | R=22.99 | EM=14.33
   Questions: 300

✓ F1=23.5688
[2/7] Running: RRF_k60_mu2000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50


RRF_k60_mu2000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50: 100%|██████████| 300/300 [06:48<00:00,  1.36s/it]



RRF_k60_mu2000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50
   Retrieval: RetrievalManager(k_docs=100, k_passages=10, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=24.40 | P=27.46 | R=24.85 | EM=14.00
   Questions: 300

✓ F1=24.3970
[3/7] Running: RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60


...

RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50: 100%|██████████| 300/300 [06:46<00:00,  1.36s/it]



RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50
   Retrieval: RetrievalManager(k_docs=100, k_passages=10, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=25.42 | P=28.56 | R=25.27 | EM=15.67
   Questions: 300

✓ F1=25.4154
[7/7] Running: RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50


RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50: 100%|██████████| 300/300 [06:40<00:00,  1.33s/it]


RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50
   Retrieval: RetrievalManager(k_docs=100, k_passages=10, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=23.77 | P=27.07 | R=23.10 | EM=13.00
   Questions: 300

✓ F1=23.7669

Grid results (sorted by F1):


,config_key,f1,precision,recall,exact_match
0,RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win250_ovl60,27.002731,30.547349,26.630051,16.333333
1,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60,25.832525,28.221849,26.169444,15.000000
2,RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50,25.415384,28.561420,25.268939,15.666667
3,RRF_k60_mu2000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50,24.396951,27.461382,24.847222,14.000000
4,RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50,23.766910,27.073655,23.102273,13.000000
5,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win150_ovl50,23.568777,27.143452,22.986111,14.333333
6,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass5_win150_ovl50,20.911268,22.670983,22.125000,11.666667


### Phase 5 - Final comparison between 2 best configs, over 1000 questions

In [27]:
PHASE_5_NUM_QUESTIONS = 1000
PHASE_5_TOP_K = 3

phase_4_path = get_experiment_log_path(num_questions=PHASE_4_NUM_QUESTIONS)
df_phase_4 = pd.read_csv(phase_4_path)

assert not df_phase_4.empty, "No Phase 4 results found"

df_phase_4 = df_phase_4.sort_values("f1", ascending=False).reset_index(drop=True)

top_3_keys = df_phase_4.head(PHASE_5_TOP_K)["config_key"].tolist()

print("Top 3 configs from Phase 4:")
for i, row in df_phase_4.head(PHASE_5_TOP_K).iterrows():
    print(f"{i+1}. {row['config_key']} | F1={row['f1']:.4f}")


PHASE_5_GRID = [
    {
        "retrieval_mgr": e["retrieval_mgr"],
        "prompt_mgr": DEFAULT_PROMPT_MANAGER,
    }
    for e in PHASE_4_GRID
    if generate_config_key(e["retrieval_mgr"], e["prompt_mgr"]) in top_3_keys
]

print(f"\n✓ Phase 5 grid size: {len(PHASE_5_GRID)}")

PHASE_5_RESULTS = run_phase(
    phase_name="PHASE 5 — Final Tie-Breaker",
    grid=PHASE_5_GRID,
    df_train=df_train,
    num_questions=PHASE_5_NUM_QUESTIONS,
    top_k=None,
)


Top 3 configs from Phase 4:
1. RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win250_ovl60 | F1=27.0027
2. RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60 | F1=25.8325
3. RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50 | F1=25.4154

✓ Phase 5 grid size: 3

PHASE 5 — Final Tie-Breaker
Questions per config: 1000
Total configs: 3
Completed: 0
Pending: 3
--------------------------------------------------------------------------------
[1/3] Running: RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60


RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60:   0%|          | 0/1000 [00:00<?, ?it/s]

RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60: 100%|██████████| 1000/1000 [38:47<00:00,  2.33s/it]



RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60
   Retrieval: RetrievalManager(k_docs=100, k_passages=10, RRF_k=60, window=250, overlap=60)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=26.89 | P=29.29 | R=28.07 | EM=15.50
   Questions: 1000

✓ F1=26.8943
[2/3] Running: RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win250_ovl60


RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win250_ovl60: 100%|██████████| 1000/1000 [39:23<00:00,  2.36s/it]



RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win250_ovl60
   Retrieval: RetrievalManager(k_docs=100, k_passages=10, RRF_k=60, window=250, overlap=60)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=26.83 | P=29.78 | R=27.45 | EM=15.40
   Questions: 1000

✓ F1=26.8328
[3/3] Running: RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50


RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50: 100%|██████████| 1000/1000 [22:42<00:00,  1.36s/it]


RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50
   Retrieval: RetrievalManager(k_docs=100, k_passages=10, RRF_k=60, window=150, overlap=50)
   Prompt: temp=0.0, top_p=1.0, max_tokens=64
   F1=26.05 | P=28.72 | R=26.29 | EM=15.90
   Questions: 1000

✓ F1=26.0505

Grid results (sorted by F1):


,config_key,f1,precision,recall,exact_match
0,RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60,26.894262,29.292044,28.068095,15.5
1,RRF_k60_mu1000_k10.6_b0.3_kdocs100_kpass10_win250_ovl60,26.832832,29.778993,27.449015,15.4
2,RRF_k60_mu2000_k10.6_b0.3_kdocs100_kpass10_win150_ovl50,26.050502,28.724577,26.287587,15.9


## Kaggle Submission

In [28]:
phase_5_path = get_experiment_log_path(num_questions=PHASE_5_NUM_QUESTIONS)
df_phase_5 = pd.read_csv(phase_5_path)

assert not df_phase_5.empty, "No Phase 5 results found"

df_phase_5 = df_phase_5.sort_values("f1", ascending=False).reset_index(drop=True)

best_row = df_phase_5.iloc[0]
BEST_CONFIG_KEY = best_row["config_key"]

print("Selected best config from Phase 5:")
print(f"{BEST_CONFIG_KEY} | F1={best_row['f1']:.4f}")

best_entry = next(
    e for e in PHASE_5_GRID
    if generate_config_key(e["retrieval_mgr"], e["prompt_mgr"]) == BEST_CONFIG_KEY
)

BEST_RETRIEVAL_MGR = best_entry["retrieval_mgr"]
BEST_PROMPT_MGR = best_entry["prompt_mgr"]

print("\nGenerating answers for test set...")

test_questions = df_test["question"].tolist()
test_ids = df_test["id"].tolist()

predictions = []

for question in tqdm(test_questions, total=len(test_questions), desc="Test Questions"):
    contexts = BEST_RETRIEVAL_MGR.retrieve_context(question)
    answer = BEST_PROMPT_MGR.generate_answer(question, contexts)
    predictions.append(answer)

submission_df = pd.DataFrame({
    "id": test_ids,
    "prediction": predictions,
})

# Required serialization for Kaggle evaluation
submission_df["prediction"] = submission_df["prediction"].apply(
    lambda x: json.dumps([x], ensure_ascii=False)
)

SUBMISSION_PATH = "./results/kaggle_submission.csv"
submission_df.to_csv(SUBMISSION_PATH, index=False)

print("=" * 80)
print(f"✓ Kaggle submission saved to: {SUBMISSION_PATH}")
print(f"✓ Total rows: {len(submission_df)}")
print("=" * 80)


Selected best config from Phase 5:
RRF_k60_mu1000_k10.9_b0.4_kdocs100_kpass10_win250_ovl60 | F1=26.8943

Generating answers for test set...


Test Questions:   0%|          | 0/2032 [00:00<?, ?it/s]

Test Questions: 100%|██████████| 2032/2032 [1:19:08<00:00,  2.34s/it]

✓ Kaggle submission saved to: ./results/kaggle_submission.csv
✓ Total rows: 2032
